In [107]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [108]:
# Setup Selenium options
options = Options()

# Do NOT use headless mode initially to see the CAPTCHA/challenge
# options.add_argument("--headless")  # You can enable this later
options.add_argument("--disable-gpu")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")

# Initialize the driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

name_list = []
date_list = []
price_list = []
shop_code_list = []
JAN_code_list = []
brand_list = []
series_title_list = []
char_name_list = []
sculptor_list = []
size_list = []
gcode_list = []
pages = 1

# while pages <= 2 :
while pages < 2:
    base_URL = "https://www.amiami.com/eng/search/list/?s_condition_flg=1&s_st_condition_flg=1&s_sortkey=preowned&pagecnt="
    final_URL = base_URL + str(pages)
    driver.get(final_URL)
    time.sleep(10)

    page_html = driver.page_source
    soup = BeautifulSoup(page_html, 'html.parser')
    items = soup.find(class_='new-items__inner').findAll('a', href=True)

    for item in items:
        gcode = item['href']  # Extract the href attribute (the URL)
        gcode_list.append(gcode)
    pages += 1

# driver.quit()
        

In [109]:
base_link = "https://www.amiami.com"
count = 0

# Cell for 1 figure only

# for gcode_url in gcode_list:
figure_url = base_link + gcode_list[1]
# doubleprice_URL = "https://www.amiami.com/eng/detail/?gcode=FIGURE-156533-R"

driver.get(figure_url)
# driver.get(doubleprice_URL)
# time.sleep(10)

figure_html = driver.page_source
driver.quit()

figure_soup = BeautifulSoup(figure_html, "html.parser")
item_details = figure_soup.findAll(class_ = ['item-detail__right', 'item-about'])

In [110]:
# price1 = figure_soup.findAll(class_ = ["item-detail__price"])
# price = figure_soup.find(class_ = ["item-detail__price"])
# temp = price
# doubleprice = temp
# print(doubleprice.text)

In [111]:
# Part of 1 figure only

# item_about = figure_soup.findAll(class_ = ["item-about__data-text"], limit = 10)
        
# temp_list = []
# for i in item_about:
#     temp_list.append(i.text)
    
# fig_name = figure_soup.find(class_ = ["item-detail__section-title"]).text
# fig_release_date = temp_list[0]
# fig_price = temp_list[1]
# fig_shop_code = temp_list[2]
# fig_JAN_code = temp_list[3]
# fig_brand = temp_list[4]
# fig_series_title = temp_list[5]
# fig_char_name = temp_list[6]
# if temp_list[7] == "The maximum purchase quantity for this item is 1 per account/shipping address.":
#     fig_sculptor = np.nan
# else:
#     fig_sculptor = temp_list[7]

In [127]:
# Dictionary Method
figures = []
figure_info = {}
count = 0
for dl in figure_soup.findAll("dl"):
    key = "Name"
    value = figure_soup.find(class_ = ["item-detail__section-title"]).text
    figure_info[key] = value
    
    for dt, dd in zip(dl.findAll("dt"), dl.findAll("dd")):
        key = dt.get_text(strip=True)
        if key == "Specifications":
            value = dd.get_text("|")            # Joins the bits of text together using | & clears out the <br/> tags
            value = value.split("|")            # Separates the long string into bits of strings so we can access stuff we want easier
            for s in value:
                if "Size" in s:
                    value = s
        else:
            value = dd.get_text(strip=True)

        if "Notice" in key or "Customs" in key or "Copyright" in key or "Details" in key:
            continue
        else:
            figure_info[key] = value
count += 1
figures.append(figure_info)

In [129]:
figures

[{'Name': '(Pre-owned ITEM:A/BOX:B)To Love-Ru Darkness - Momo Belia Deviluke Yukata Ver. 1/8 Complete Figure(Released)',
  'Release Date': 'Sep-2015',
  'List Price': '11,000 JPY',
  'Shop Code': 'FIGURE-011682-R105',
  'JAN code': '4571245295163',
  'Brand': 'FREEing',
  'Product Line': 'Y-STYLE',
  'Series Title': 'To Love-Ru Darkness"To Love-Ru" Series',
  'Character Name': 'Momo Belia Deviluke',
  'Sculptor': 'KANAHeavy Gauge',
  'Specifications': 'Size: Approx. H200mm',
  'Remarks': 'For sale Japan only'}]

In [120]:
figure_df = pd.DataFrame(figures)
figure_df

,Release Date,List Price,Shop Code,JAN code,Brand,Product Line,Series Title,Character Name,Sculptor,Specifications,Remarks
0,Sep-2015,"11,000 JPY",FIGURE-011682-R105,4571245295163,FREEing,Y-STYLE,"To Love-Ru Darkness""To Love-Ru"" Series",Momo Belia Deviluke,KANAHeavy Gauge,Size: Approx. H200mm,For sale Japan only


In [ ]:
item_specs = figure_soup.find(class_ = ["more"])
        
specs = item_specs.get_text("|")    # Joins the bits of text together using | & clears out the <br/> tags
splitSpecs = specs.split("|")       # Separates the long string into bits of strings so we can access stuff we want easier
flag = 0
for s in splitSpecs:
    print(s)
    if flag != 1:
        if "Size" in s:
            # fig_size = ""
            fig_size = s
            flag = 1
        else:
            fig_size = temp_list[8]

In [ ]:
name_list.append(fig_name)
date_list.append(fig_release_date)
price_list.append(fig_price)
shop_code_list.append(fig_shop_code)
JAN_code_list.append(fig_JAN_code)
brand_list.append(fig_brand)
series_title_list.append(fig_series_title)
char_name_list.append(fig_char_name)
sculptor_list.append(fig_sculptor)
size_list.append(fig_size)

In [ ]:
# For more than 1 figure

# for gcode_url in gcode_list:
#     # print(count)
#     # try:
#     figure_url = base_link + gcode_list[count]
#     # print(figure_url)
#     driver.get(figure_url)
#     time.sleep(5)                       # Unsure if I need to add a sleep function here to prevent overloading the server. Referenced example has a waitTime variable of 5. 
#     figure_html = driver.page_source
#     # driver.quit()
#     try:
#         figure_soup = BeautifulSoup(figure_html, "html.parser")
#         item_details = figure_soup.findAll(class_ = ['item-detail__right', 'item-about'])
#         item_about = figure_soup.findAll(class_ = ["item-about__data-text"], limit = 11)
        
#         temp_list = []
#         for i in item_about:
#             temp_list.append(i.text)
            
#         fig_name = figure_soup.find(class_ = ["item-detail__section-title"]).text
#         fig_release_date = temp_list[0]
#         fig_price = temp_list[1]
#         fig_shop_code = temp_list[2]
#         fig_JAN_code = temp_list[3]
#         fig_brand = temp_list[4]
#         fig_series_title = temp_list[5]
#         fig_char_name = temp_list[6]
#         if temp_list[7] == "The maximum purchase quantity for this item is 1 per account/shipping address.":
#             fig_sculptor = np.nan
#         else:
#             fig_sculptor = temp_list[7]    
        
#         item_specs = figure_soup.find(class_ = ["more"])
        
#         specs = item_specs.get_text("|")    # Joins the bits of text together using | & clears out the <br/> tags
#         splitSpecs = specs.split("|")       # Separates the long string into bits of strings so we can access stuff we want easier
#         flag = 0
#         for s in splitSpecs:
#             if flag != 1:
#                 if "Size" in s:
#                     fig_size = s
#                     flag = 1               
#                     # if "Please confirm the following conditions of pre-owned items before placing order" not in temp_list[9]:
#                     #     fig_size = temp_list[9]
#                     # # elif "The maximum purchase quantity for this item is 1 per account/shipping address." not in temp_list[8]:
#                     # else:
#                     #     temp_list[8]
        
#         if flag == 0:
#             for temp_item in temp_list:
#                 if flag == 0:
#                     if "Size:" in temp_item or "Scale:" in temp_item:
#                         fig_size = temp_item
#                         flag = 1
#             if flag == 0:
#                 fig_size = "Error"

                
        
#         name_list.append(fig_name)
#         date_list.append(fig_release_date)
#         price_list.append(fig_price)
#         shop_code_list.append(fig_shop_code)
#         JAN_code_list.append(fig_JAN_code)
#         brand_list.append(fig_brand)
#         series_title_list.append(fig_series_title)
#         char_name_list.append(fig_char_name)
#         sculptor_list.append(fig_sculptor)
#         size_list.append(fig_size)
#         count += 1

#     except:
#         print(figure_url)
#         count += 1
#     # print(figure_url)

In [9]:
driver.quit()

In [10]:
df_figure = pd.DataFrame(
    {'Name': name_list,
     'Release Date': date_list,
     'Price': price_list,
     'Shop Code': shop_code_list,
     'JAN Code': JAN_code_list,
     'Brand': brand_list,
     'Series Title': series_title_list,
     'Character Name': char_name_list,
     'Sculptor': sculptor_list,
     'Size Specifications': size_list
    })

fig_grade = ['A', 'A\-', 'B\+', 'B', 'C', 'J']      # "+" & "-" also included as regex expressions. "\" to match literal characters
box_grade = ['A', 'B', 'C', 'N']
bonus_grade = ['\[Bonus\]', '']
product_grade = [fr'\(Pre-owned ITEM:{fig}/BOX:{box}\){bonus}' for fig in fig_grade for box in box_grade for bonus in bonus_grade]

df_figure["Name"] = df_figure["Name"].replace(to_replace=product_grade, value='', regex=True)

In [14]:
brands = df_figure.groupby("Release Date")
brands.first()

,Name,Price,Shop Code,JAN Code,Brand,Series Title,Character Name,Sculptor,Size Specifications
Release Date,,,,,,,,,
Apr-2010,figma - Magical Girl Lyrical Nanoha StrikerS: ...,"2,934 JPY",FIG-MOE-1473-R238,4545784061169,Masaki Apsy - Max Factory,The maximum purchase quantity for this item is...,ABS & PVC Pre-Painted Posable FigureSize: Appr...,Sculptor: Masaki Apsy - Max FactoryThe front a...,Size: Approx. H140mm (NON Scale)
Aug-2024,POP UP PARADE Hololive Production Houshou Mari...,"5,500 JPY",FIGURE-167440-R014,4580416949712,Good Smile Company,POP UP PARADE,Hololive[Virtual YouTuber],Houshou Marine,Size: Approx. H170mm (non-scale)
Dec-2021,Nendoroid The Quintessential Quintuplets Nino ...,"5,200 JPY",FIGURE-126967-R111,4580590124769,The Quintessential Quintuplets,Nino Nakano,Design COCO,None,Size: Approx. H100mm (non-scale)
Feb-2018,Nendoroid - Snow Miku Crane Priestess Ver. [SN...,"6,112 JPY",FIGURE-036482-R293,4580416904568,Good Smile Company,"""Nendoroid"" SeriesNendoroid","""VOCALOID"" Series",Hatsune Miku,Size: Approx. H100mm (non-scale)
Feb-2021,Sakura Miku New Illustration Figure 2021 ver. ...,Open Price,FIGURE-119862-R188,,Taito,"""VOCALOID"" Series",Hatsune Miku,None,Size: Approx. 18cm
Feb-2023,Nendoroid Character Vocal Series 01 Snow Miku ...,"9,000 JPY",FIGURE-149844-R061,4580590171701,Good Smile Company,"""Nendoroid"" SeriesNendoroid","""VOCALOID"" Series",Hatsune Miku,Size: Approx. H100mm (non-scale)
Jan-2024,Blue Archive Kakudate Karin 1/7 Complete Figur...,"29,800 JPY",FIGURE-151289-R075,4545784043547,Max Factory,Blue Archive,Karin(Kakudate Karin),CKB/Max Factory,Size: Approx. H265mm
Jun-2013,Nendoroid - Girls und Panzer: Miho Nishizumi(R...,"3,981 JPY",FIG-MOE-8345-R499,4571368443038,Good Smile Company,"""Nendoroid"" SeriesNendoroid","Girls und Panzer""Girls und Panzer"" Series",Miho Nishizumi,Size: Approx. H100mm (non-scale)
Jun-2016,Character Vocal Series 01. Hatsune Miku Greate...,"12,222 JPY",FIGURE-013610-R392,4571368442567,Good Smile Company,"""VOCALOID"" Series",Hatsune Miku,ATOMICBOM SUZUKI,Size: Approx. H200mm
